In [1]:
def initialize_domains():
    domains = []  # domain for each cell
    dom_sizes = []  # size of the domain for each cell
    for i in range(9):
        domains.append([])
        dom_sizes.append([])
        for j in range(9):
            domains[i].append([])
            dom_sizes[i].append(9)
            for k in range(9):
                domains[i][j].append(True)
    return domains, dom_sizes

In [2]:
def read_data(filename):
    with open(filename) as f:
        data = f.readlines()

    board = []
    domains, dom_sizes = initialize_domains()

    for i, line in enumerate(data):
        line = line.strip()  # remove newline character
        line = line.split("|")[1:-1]
        row = []
        for j, cell in enumerate(line):
            if cell == " ":
                row.append(0)  # 0 represents an empty cell
            else:
                cell = int(cell)
                row.append(cell)
                domains, dom_sizes, consistent = consistency_check(domains, dom_sizes, i, j, cell)
                if not consistent:
                    raise ValueError("invalid input board")
        board.append(row)
    
    return board, domains, dom_sizes

In [3]:
def display_board(board):
    for row in board:
        print("|", end="")
        for cell in row:
            print(cell, end="|")
        print()

In [4]:
import copy

In [5]:
def consistency_check(domains, dom_sizes, i, j, k):
    domains = copy.deepcopy(domains)
    dom_sizes = copy.deepcopy(dom_sizes)
    
    # update domain for cell (i,j)
    domains[i][j] = [False] * 9
    domains[i][j][k-1] = True
    dom_sizes[i][j] = 1
    
    # update domains for cells in same column
    for x in range(9):
        y = j
        if x != i:
            if domains[x][y][k-1]:
                dom_sizes[x][y] -= 1
                if dom_sizes[x][y] == 0:
                    return None, None, False
            domains[x][y][k-1] = False
    
    # update domains for cells in same row
    for y in range(9):
        x = i
        if y != j:
            if domains[x][y][k-1]:
                dom_sizes[x][y] -= 1
                if dom_sizes[x][y] == 0:
                    return None, None, False
            domains[x][y][k-1] = False
            
    # update domains for cells in same subgrid
    r = i // 3
    c = j // 3
    for x in range(3*r, 3*(r+1)):
        for y in range(3*c, 3*(c+1)):
            if x != i and y != j:
                if domains[x][y][k-1]:
                    dom_sizes[x][y] -= 1
                    if dom_sizes[x][y] == 0:
                        return None, None, False
                domains[x][y][k-1] = False
    
    return domains, dom_sizes, True

In [6]:
def backtrack(cell_id, board, domains, dom_sizes):
    if cell_id >= 81:
        # Sudoku puzzle is completed
        return True, board, domains, dom_sizes
    
    i = cell_id // 9
    j = cell_id % 9
    
    board = copy.deepcopy(board)
    domains = copy.deepcopy(domains)
    dom_sizes = copy.deepcopy(dom_sizes)
    
    for k, val in enumerate(domains[i][j]):
        if val:
            # try a new value in cell (i,j)
            board[i][j] = k+1
            new_domains, new_dom_sizes, consistent = consistency_check(domains, dom_sizes, i, j, k+1)
            
            if consistent:
                # move on to next cell
                res, new_board, new_domains, new_dom_sizes = backtrack(cell_id + 1, board, new_domains, new_dom_sizes)
                if res:
                    # Sudoku puzzle is completed
                    return True, new_board, new_domains, new_dom_sizes
    
    # no values worked for cell (i,j); going back to previous cell
    return False, board, domains, dom_sizes

In [7]:
def solve(board, domains, dom_sizes):
    res, board, domains, dom_sizes = backtrack(0, board, domains, dom_sizes)
    if res:
        print("Here is the solution to this Sudoku puzzle:\n")
        display_board(board)
    else:
        print("There is no solution to this Sudoku puzzle.")

In [8]:
filename = "input_boards/sample_board.txt"
board, domains, dom_sizes = read_data(filename)
display_board(board)

|8|0|0|9|3|0|0|0|2|
|0|0|9|0|0|0|0|4|0|
|7|0|2|1|0|0|9|6|0|
|2|0|0|0|0|0|0|9|0|
|0|6|0|0|0|0|0|7|0|
|0|7|0|0|0|6|0|0|5|
|0|2|7|0|0|8|4|0|6|
|0|3|0|0|0|0|5|0|0|
|5|0|0|0|6|2|0|0|8|


In [9]:
from time import perf_counter

In [10]:
start = perf_counter()
solve(board, domains, dom_sizes)
end = perf_counter()
print(f"\ntime elapsed (s): {end - start}")

Here is the solution to this Sudoku puzzle:

|8|4|6|9|3|7|1|5|2|
|3|1|9|6|2|5|8|4|7|
|7|5|2|1|8|4|9|6|3|
|2|8|5|7|1|3|6|9|4|
|4|6|3|8|5|9|2|7|1|
|9|7|1|2|4|6|3|8|5|
|1|2|7|5|9|8|4|3|6|
|6|3|8|4|7|1|5|2|9|
|5|9|4|3|6|2|7|1|8|

time elapsed (s): 0.829712800001289
